<a href="https://colab.research.google.com/github/AndreRFlenik/Analise_De_Concorrencia_Mercado_Livre/blob/main/Analise_de_Concorr%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de Bibliotecas


In [ ]:
# Importar as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

import statsmodels.api as sm
import plotly.express as px


from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler


#Função de Busca no Mercado Livre - WebScraping

In [ ]:
def search_mercadolivre(product_name):
    url = f"https://lista.mercadolivre.com.br/{product_name}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.find_all('li', {'class': 'ui-search-layout__item'})

    datasearch = []

    for product in products:
        name_element = product.find('h2', {'class': 'ui-search-item__title'})
        name = name_element.text.strip() if name_element else 'Nome não disponível'

        price_element = product.find('span', {'class': 'andes-money-amount'})
        if price_element:
            price_text = price_element.text.strip()
            price_text = price_text.replace('R$', '').replace('.', '').replace(',', '.').strip()
            price = float(price_text)
        else:
            price = 0.0

        avaliation_element = product.find('span', {'class': 'ui-search-reviews__rating-number'})
        avaliation = avaliation_element.text.strip() if avaliation_element else '0'
        avaliation = float(avaliation)

        opinion_element = product.find('span', {'class': 'ui-search-reviews__amount'})
        if opinion_element:
            opinion_text = opinion_element.text.strip()
            opinion_match = re.search(r'\((\d+)\)', opinion_text)
            opinion = int(opinion_match.group(1)) if opinion_match else 0
        else:
            opinion = 0
        opinion = float(opinion)

        link_element = product.find('a', {'class': 'ui-search-link'})
        link = link_element['href'] if link_element else 'Link não disponível'

        # Nova parte para extrair o número de vendas
        sales = 0
        if link != 'Link não disponível':

            # Requisição para a página do produto
            product_response = requests.get(link)
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            # Encontrar o elemento que contém as vendas
            sales_element = product_soup.find('span', {'class': 'ui-pdp-subtitle'})
            if sales_element:
                # Extrair o número de vendas usando expressões regulares
                sales_match = re.search(r'\+(\d+)\s+vendidos', sales_element.text)
                sales = int(sales_match.group(1)) if sales_match else 0
                sales= float(sales)

                 # Verificar categoria Full
            full_category = 0
            full_icon = product_soup.find('svg', {'class': 'ui-pdp-icon ui-pdp-icon--full ui-pdp-color--GREEN'})
            if full_icon:
                full_category = 1

        datasearch.append([name, price, avaliation, opinion, sales, full_category])

    return datasearch

#Busca dos Produtos Concorrentes

##Inserção de dados do produto do usuário

In [ ]:
# Insira os dados do Produto (Nome, Preço Estimado, se é Full)
produto_nome = 'Kit 10 Peças Rolamento 6002 Zz Medida 15x32x9'
produto_preco = [20]
produto_full =  [1]

#OBS.:Quanto mais detalhado o nome do produto maior será a precisão da busca.

## Dataframe com todos os produtos Encontrados

In [ ]:
# Criação do DataFrame com os produtos concorrentes
df_pesquisa = pd.DataFrame(columns=['produto_nome','produto_preco', 'produto_nota','produto_votantes','produto_vendas', 'produto_full'])

# Chamar a função para preencher o dataframe
data = search_mercadolivre(produto_nome)
df_pesquisa = pd.concat([df_pesquisa, pd.DataFrame(data, columns=['produto_nome','produto_preco', 'produto_nota','produto_votantes','produto_vendas', 'produto_full'])], ignore_index=True)


df_pesquisa.head()

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full
0,Kit 10 Peças Rolamento 6002 Zz Medida 15x32x9,17.26,4.7,20.0,500.0,0
1,Kit 10 Peças Rolamento 6002 Zz Medida 15x32x9,35.90,0.0,0.0,50.0,1
2,Kit 10 Peças Rolamento Skf 6002 Zz Original 15...,199.90,5.0,1.0,5.0,0
3,Kit 10 Peças Rolamento Skf 6002 Zz/c3 Original...,199.90,0.0,0.0,0.0,0
4,Kit 10 Peças Rolamento 6002 Zz Medida 15x32x9,38.56,0.0,0.0,0.0,0


# Clusterização dos Produtos Encontrados


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px

# Remover a coluna 'produto_nome' do DataFrame para permenecer apenas as variaveis numéricas
df_cluster = df_pesquisa.drop(columns=['produto_nome'])

# Escalar as variáveis
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Instanciar o modelo KMeans com o número desejado de clusters
num_clusters = 3  # Três grupos selecionados
kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)

# Ajustar o modelo aos dados escalados
kmeans_model.fit(df_scaled)

# Adicionar rótulos de cluster aos dados
df_cluster['Cluster'] = kmeans_model.labels_

# Adicionar a coluna 'Cluster' ao DataFrame df_pesquisa
df_pesquisa = df_pesquisa.merge(df_cluster[['Cluster']], left_index=True, right_index=True)

# Plotar o gráfico de dispersão com cores diferentes para cada cluster
fig = px.scatter(df_cluster, x='produto_preco', y='produto_nota', color='Cluster', title='Clusters de Produtos')
fig.show()



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



##Verificar o Cluster em que o produto pertence

In [ ]:
# Tratamento das variáveis
produto_preco = produto_preco[0]  # Acessa o primeiro elemento da lista, se for o caso
produto_full = produto_full[0]  # Acessa o primeiro elemento da lista, se for o caso

# Dados do novo produto
novo_produto = {
    'produto_nome': [produto_nome],
    'produto_preco': [produto_preco],
    'produto_full': [produto_full]
}

# Converter para DataFrame
df_novo_produto = pd.DataFrame(novo_produto)

# Selecionar as mesmas variáveis que foram usadas para o treinamento
variaveis = ['produto_preco', 'produto_full']

# Verificar se as variáveis estão presentes no DataFrame df_pesquisa
if not all(var in df_pesquisa.columns for var in variaveis):
    raise ValueError("As variáveis não estão presentes no DataFrame df_pesquisa.")

# Normalizar os dados
scaler = StandardScaler()
df_treinamento = df_pesquisa[variaveis]
df_treinamento_scaled = scaler.fit_transform(df_treinamento)

# Normalizar o novo produto
df_novo_produto[variaveis] = df_novo_produto[variaveis].astype(float)  # Converter para tipo float, se necessário
df_novo_produto_scaled = scaler.transform(df_novo_produto[variaveis])

# Treinar o modelo K-means com os dados existentes
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_treinamento_scaled)

# Prever o cluster do novo produto
cluster_novo_produto = kmeans.predict(df_novo_produto_scaled)

print(f'O novo produto pertence ao cluster: {cluster_novo_produto[0]}')





O novo produto pertence ao cluster: 1


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [ ]:

cluster_desejado = cluster_novo_produto[0]

# Apenas os produtos do cluster pesquisado
if 'Cluster' in df_pesquisa.columns:
    df_produtos_cluster = df_pesquisa[df_pesquisa['Cluster'] == cluster_desejado].copy()

    df_produtos_cluster

else:
    print("A coluna 'Cluster' não está presente no DataFrame df_pesquisa.")

df_produtos_cluster

,produto_nome,produto_preco,produto_nota,produto_votantes,produto_vendas,produto_full,Cluster
12,Kit 10 Peças Rolamento 608 Zz Medida 8x22x7,17.66,4.7,89.0,1000.0,1,1
